In [1]:
import psutil

# Get total, used, and available memory in GB
mem = psutil.virtual_memory()
print(f"Total RAM: {mem.total / 1e9:.2f} GB")
print(f"Available RAM: {mem.available / 1e9:.2f} GB")
print(f"Used RAM: {mem.used / 1e9:.2f} GB")
print(f"RAM Usage: {mem.percent}%")

!nvidia-smi

Total RAM: 201.34 GB
Available RAM: 188.80 GB
Used RAM: 10.70 GB
RAM Usage: 6.2%
Wed Jul 23 10:53:48 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:3B:00.0 Off |                    0 |
|  0%   27C    P8    12W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  

In [2]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"Find {gpu_count} GPU can be used.")

    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("No GPU can be used.")


Find 3 GPU can be used.
GPU 1: NVIDIA A40
GPU 2: NVIDIA A40
GPU 3: NVIDIA A40


In [3]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
try:
    from rouge import Rouge
except:
    !pip install rouge
    from rouge import Rouge
# from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm

torch.set_grad_enabled(False)
tqdm.pandas()

In [4]:
import sys
sys.version

'3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]'

In [8]:
torch.cuda.set_device(1)

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join

model_dir = "../models"
# model_name = 'Meta-Llama-3-2B'  #gemma-2-9b-it  gemma-2-9b
# model_name = 'Llama-3.2-1B'
model_name = 'DeepSeek-R1-Distill-Qwen-1.5B'
output_dir = '../outputs'
dataset_dir = '../dataset'

#'Meta-Llama-3-8B-Instruct' #'Llama-2-7b-chat-hf' #'llama-7b' 
# "OLMo-7B-Instruct" olmo-2-1124-7B-Instruct 'OLMo-2-1124-7B'  "OLMo-7B-0724-Instruct-hf" olmo1的情况都非常反常，不知道为什么
# Qwen-7B, Qwen1.5-7B, Qwen-7B-Chat, Qwen1.5-7B-Chat, Qwen2-7B-Instruct, Qwen2.5-7B-Instruct, Qwen2.5-Coder-7B, Qwen2.5对中文做过优化，情况不太一样
# Mistral-7B-Instruct-v0.1, Mistral-7B-Instruct-v0.2, Mistral-7B-Instruct-v0.3, Mistral-7B-v0.3
# Yi-1.5-6B-Chat, Yi-6B-Chat
# gemma-2-9b-it gemma-7b-it, gemma-2-9b
# gpt-j-6b, mpt-7b-chat, opt-6.7b, pythia-6.9b, zephyr-7b-beta, falcon-7b-instruct
# deepseek


model = AutoModelForCausalLM.from_pretrained(
    join(model_dir, model_name),
    torch_dtype=torch.float32,
    trust_remote_code=True,
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name), trust_remote_code=True)

if 'llama' in model.config.model_type.lower() or 'mistral' in model.config.model_type.lower() or 'yi' in model.config.model_type.lower() or 'gptj' in model.config.model_type.lower():
    tokenizer.pad_token_id = tokenizer.eos_token_id
# elif 'qwen' in model.config.model_type.lower(): # not sure if this needs to be modified for Deepseek-R1
    tokenizer.pad_token = '<|endoftext|>'
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    # in gemma, pad_token_id = 0 is default
    # in olmo, pad_token_id = 1 is default
    
    
tokenizer.padding_side = "left"

model.to('cuda');

In [9]:
print('model: ',model)
print('model.config: ',model.config)
print('model.config.model_type.lower(): ',model.config.model_type.lower())  # Often provides a string identifier
# print('model.config.num_hidden_layers: ',model.config.num_hidden_layers)


if 'gptj' in model.config.model_type.lower():
    model_layers_num = int(model.config.n_layer)  
    mlp_vector_num = 16384
    mlp_dim_num = int(model.config.n_embd)
    layer_name = 'transformer.h'
    mlp_name = 'mlp'
    mlp_last_layer_name = 'fc_out'

elif 'qwen' in model.config.model_type.lower() and 'qwen2' not in model.config.model_type.lower(): #qwen1, qwen2.5
    layer_name = 'transformer.h'
    mlp_name = 'mlp'
    mlp_last_layer_name = 'w2'
    mlp_dim_num = int(model.config.hidden_size)
    model_layers_num = int(model.config.num_hidden_layers)
    mlp_vector_num = int(model.config.intermediate_size / 2)
    
else:
    model_layers_num = int(model.config.num_hidden_layers)  # on olmo1, olmo2, qwen2, qwen2.5, llama, ...
    mlp_vector_num = int(model.config.intermediate_size)
    mlp_dim_num = int(model.config.hidden_size)
    layer_name = 'model.layers' 
    mlp_name = 'mlp'
    mlp_last_layer_name = 'down_proj'
    attn_name = 'self_attn'

model:  Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_

In [11]:
import datasets
import json
import re
import random
from datasets import load_from_disk
from tqdm import tqdm

n_new_tokens = 100
NUll_num = 0

def form_options(options: list):
    option_str = 'Options are:\n'
    opts = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    for opt, o in zip(options, opts):
        option_str += f'({o}): {opt}' + '\n'
    return option_str


def get_prediction(output):
    pattern = r"answer is \(?([ABCDEFGHIJ])\)?"
    match = re.search(pattern, output)
    if match:
        #print('prediction success: ',match.group(1))
        return match.group(1)
    else:
        #print("extraction failed, do a random guess")
        global NUll_num  
        NUll_num += 1
        return random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])


def generate_outputs(questions):
    
    inputs = tokenizer(questions, return_tensors="pt", padding="longest", return_token_type_ids=False).to('cuda')
    input_length = inputs.input_ids.size(1)
    output = model(**inputs, output_hidden_states = True)
#     Question_input = [[{"role": "user", "content": prompt}] for prompt in questions]
#     texts = tokenizer.apply_chat_template(Question_input ,tokenize=False)

#     inputs = tokenizer(texts, padding="longest", return_tensors="pt")
#     inputs = {key: val.cuda() for key, val in inputs.items()}
#     output = model(**inputs, output_hidden_states = True)
    
    return output

def generate_questions(questions):
    
    inputs = tokenizer(questions, return_tensors="pt", padding="longest", return_token_type_ids=False).to('cuda')
    input_length = inputs.input_ids.size(1)
    gen_tokens = model.generate(**inputs, max_new_tokens=n_new_tokens, do_sample=False)

    gen_text = tokenizer.batch_decode(gen_tokens[:, input_length:], skip_special_tokens=True)
    
    return gen_text


dataset = load_from_disk('../dataset/mmlu-pro')

categories = ['computer science', 'math', 'chemistry', 'engineering', 'law', 'biology',
              'health', 'physics', 'business', 'philosophy', 'economics', 'other',
              'psychology', 'history']

per_category_accuracy = {c: [0, 0] for c in categories}
success, fail = 0, 0
answers = []

print('----------------- Start Answering -------------------')
queries_batch = []  # 可以测试一下batch or single哪种方式准确率更高，更合适一些 #发现基本是一样的，padding不会对准确率造成影响
entry_batch = []

batch_size = 4

random.seed(8888)

test_data = list(dataset['test'])

# example: running on 600 samples
sampled_data_600 = random.sample(test_data, 600)

layers_to_cache = list(range(model_layers_num))
print('layers_to_cache: ',layers_to_cache)
hs_cache_cot = {}
hs_cache_no_cot = {}

print('----------------- Running no cot Inference -------------------')
for ix, entry in tqdm(enumerate(sampled_data_600)):
        
    query = 'Q: ' + entry['question'] + "\nA: "
    
    queries_batch.append(query)
    
    if len(queries_batch) == batch_size or ix == len(dataset['test']) - 1:
        output = generate_outputs(queries_batch)
        
        for layer in layers_to_cache:
            if layer not in hs_cache_no_cot:
                hs_cache_no_cot[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
            else:
                hs_cache_no_cot[layer] = torch.cat((hs_cache_no_cot[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

        
        queries_batch = []
    torch.cuda.empty_cache()
# # TODO print('----------------- Running cot Inference -------------------')

sampled_data_3000 = random.sample(test_data, 3000)

layers_to_cache = list(range(model_layers_num))
print('layers_to_cache: ',layers_to_cache)
# hs_cache_cot = {}
hs_cache_no_cot_3000 = {}

print('----------------- Running no cot Inference (3000 samples MMLU-pro) -------------------')
for ix, entry in tqdm(enumerate(sampled_data_3000)):
        
    query = 'Q: ' + entry['question'] + "\nA: "
    
    queries_batch.append(query)
    
    if len(queries_batch) == batch_size or ix == len(dataset['test']) - 1:
        output = generate_outputs(queries_batch)
        
        for layer in layers_to_cache:
            if layer not in hs_cache_no_cot_3000:
                hs_cache_no_cot_3000[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
            else:
                hs_cache_no_cot_3000[layer] = torch.cat((hs_cache_no_cot_3000[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

        
        queries_batch = []
    torch.cuda.empty_cache()


----------------- Start Answering -------------------
layers_to_cache:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
----------------- Running no cot Inference -------------------


600it [00:20, 29.98it/s]


# **PCA**

In [12]:
import os
import json
import pandas as pd

In [30]:
# loading and running gsm8k or other dataset


gsm8k_ds_main = load_from_disk('../dataset/gsm8k/main') 
gsm8k_ds_main_test = list(gsm8k_ds_main['test'])

mbpp_ds_full = load_from_disk('../dataset/mbpp/full')
mbpp_ds_full_val = list(mbpp_ds_full['validation'])

mbpp_ds_full = load_from_disk('../dataset/mbpp/full')
mbpp_ds_full_test = list(mbpp_ds_full['test'])

# example on MGSM, feel free to add other categories in MGSM
mgsm_zh = pd.read_csv('../dataset/mgsm/mgsm_zh.tsv', sep='\t', header=None)
mgsm_zh_test = mgsm_zh.values.tolist()
mgsm_de = pd.read_csv('../dataset/mgsm/mgsm_de.tsv', sep='\t', header=None)
mgsm_de_test = mgsm_de.values.tolist()
mgsm_bn = pd.read_csv('../dataset/mgsm/mgsm_bn.tsv', sep='\t', header=None)
mgsm_bn_test = mgsm_bn.values.tolist()
mgsm_ja = pd.read_csv('../dataset/mgsm/mgsm_ja.tsv', sep='\t', header=None)
mgsm_ja_test = mgsm_ja.values.tolist()
mgsm_te = pd.read_csv('../dataset/mgsm/mgsm_te.tsv', sep='\t', header=None)
mgsm_te_test = mgsm_te.values.tolist()


# example on C-Eval, feel free to add other categories in C-Eval
ceval_chi = pd.read_csv('../dataset/ceval-exam/test/chinese_language_and_literature_test.csv')['question']
ceval_chi_test = ceval_chi.tolist()
ceval_his = pd.read_csv('../dataset/ceval-exam/test/high_school_history_test.csv')['question']
ceval_his_test = ceval_his.tolist()
ceval_pol = pd.read_csv('../dataset/ceval-exam/test/high_school_politics_test.csv')['question']
ceval_pol_test = ceval_pol.tolist()
ceval_mar = pd.read_csv('../dataset/ceval-exam/test/marxism_test.csv')['question']
ceval_mar_test = ceval_mar.tolist()
ceval_bus = pd.read_csv('../dataset/ceval-exam/test/business_administration_test.csv')['question']
ceval_bus_test = ceval_bus.tolist()

popqa_test = load_from_disk('../dataset/PopQA/test') 
popqa_test = list(popqa_test)


def load_jsonl_file(jsonl_path, max_samples=None):
    """Loads .jsonl files line by line."""
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    data = [json.loads(line) for line in lines]
    return data if max_samples is None else data[:max_samples]

def load_gsm_symbolic_data(base_dir, max_samples_per_file=200):
    """Loads multiple JSONL files from gsm_symbolic_data."""
    combined = []
    for fname in ["GSM_p1.jsonl", "GSM_p2.jsonl", "GSM_symbolic.jsonl"]:
        path = os.path.join(base_dir, fname)
        if os.path.exists(path):
            combined.extend(load_jsonl_file(path, max_samples_per_file))
    return combined

# --- Paths (adjust if needed) ---
DATASET_ROOT = "../dataset"

gsm_sym_data = load_gsm_symbolic_data(os.path.join(DATASET_ROOT, "gsm_symbolic_data"))
human_eval_path = os.path.join(DATASET_ROOT, "HumanEval", "HumanEval.jsonl")
human_eval_data = load_jsonl_file(human_eval_path, max_samples=600)

print(f"Loaded {len(gsm_sym_data)} samples from GSM Symbolic")
print(f"Loaded {len(human_eval_data)} samples from HumanEval")


other_running_set_name_list = ['ceval_liberal', 'gsm8k', 'mgsm', 'popqa', 'gsm_symbolic', 'human_eval', 'human_eval', 'mbpp'] # mbpp,, hoppingtoolate， , 'mbpp', 'popqa',
# other_running_set_name_list = [ 'gsm_symbolic', 'human_eval', 'human_eval', 'mbpp']
# other_running_set_name_list = ['ceval_liberal', 'gsm8k', 'mgsm', 'popqa'] # mbpp,, hoppingtoolate， , 'mbpp', 'popqa',


other_dataset = None

hs_cache_no_cot_other_all = {}

for other_running_set_name in other_running_set_name_list:
    
    if other_running_set_name == 'mbpp':
        other_dataset = mbpp_ds_full_test
         
    elif other_running_set_name == 'gsm8k':
        other_dataset = gsm8k_ds_main_test
         
    elif other_running_set_name == 'mgsm': #multilingual gsm8k
        other_dataset = mgsm_zh_test + mgsm_de_test + mgsm_bn_test + mgsm_ja_test + mgsm_te_test
         
    elif other_running_set_name == 'ceval_liberal':
        other_dataset = ceval_chi_test + ceval_his_test + ceval_pol_test + ceval_mar_test # + ceval_bus_test
         
    elif other_running_set_name == 'popqa': #multilingual gsm8k
        other_dataset = popqa_test
         
    elif other_running_set_name == 'gsm_symbolic':
        other_dataset = gsm_sym_data
         
    elif other_running_set_name == 'human_eval':
        other_dataset = human_eval_data
         
    elif other_running_set_name == 'gsm_symbolic':
        other_dataset = gsm_sym_data
         


    print(f'#####Running on {other_running_set_name} test set')
    print(f'The size is {len(other_dataset)}')

    layers_to_cache_other = list(range(model_layers_num))
    print('layers_to_cache_other: ',layers_to_cache_other)
    hs_cache_no_cot_other = {}
    queries_batch = []
    batch_size = 4

    for ix, entry in tqdm(enumerate(other_dataset)):

        if other_running_set_name == 'gsm8k':
            query = 'Q: ' + entry['question'] + "\nA: "
        elif other_running_set_name == 'mbpp':
            query = 'Q: ' + entry['text'] + "\nA: "
        elif other_running_set_name == 'mgsm':
            query = 'Q: ' + entry[0] + "\nA: "
        elif other_running_set_name == 'ceval_liberal':
            query = 'Q: ' + entry + "\nA: "
        elif other_running_set_name == 'popqa':
            query = 'Q: ' + entry['question'] + "\nA: "

        queries_batch.append(query)

        if len(queries_batch) == batch_size or ix == len(other_dataset) - 1:
            output = generate_outputs(queries_batch)

            for layer in layers_to_cache_other:
                if layer not in hs_cache_no_cot_other:
                    hs_cache_no_cot_other[layer] = output["hidden_states"][layer][: ,-1 , :].cpu() #bs * tok * dims
                else:
                    hs_cache_no_cot_other[layer] = torch.cat((hs_cache_no_cot_other[layer], output["hidden_states"][layer][: ,-1 , :].cpu()), dim=0)

            queries_batch = []
        torch.cuda.empty_cache()

    hs_cache_no_cot_other_all[other_running_set_name] = hs_cache_no_cot_other


Loaded 600 samples from GSM Symbolic
Loaded 164 samples from HumanEval
#####Running on ceval_liberal test set
The size is 746
layers_to_cache_other:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


746it [00:14, 49.77it/s]


#####Running on gsm8k test set
The size is 1319
layers_to_cache_other:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


1319it [00:33, 39.24it/s]


#####Running on mgsm test set
The size is 1250
layers_to_cache_other:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


...

164it [00:01, 110.06it/s]


#####Running on human_eval test set
The size is 164
layers_to_cache_other:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


164it [00:01, 101.30it/s]


#####Running on mbpp test set
The size is 500
layers_to_cache_other:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


500it [00:06, 81.53it/s]


In [31]:
hs_cache_no_cot_other_all['mmlu-pro_600'] = hs_cache_no_cot
hs_cache_no_cot_other_all['mmlu-pro_3000'] = hs_cache_no_cot_3000

In [16]:
pd.DataFrame(other_dataset)#

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,4725190,John Mayne,occupation,journalist,2079053,22,663400,[],"[""journo"",""journalists""]",http://www.wikidata.org/entity/Q6247345,http://www.wikidata.org/entity/Q1930187,John Mayne,Journalist,236,24952,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,4382392,Henry Feilden,occupation,politician,1925450,22,2834605,"[""Henry Master Feilden""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5725578,http://www.wikidata.org/entity/Q82955,Henry Feilden (Conservative politician),Politician,58,25692,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,4822110,Kathy Saltzman,occupation,politician,2122743,22,2834605,[],"[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q6377295,http://www.wikidata.org/entity/Q82955,Kathy Saltzman,Politician,127,25692,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,4011112,Eleanor Davis,occupation,cartoonist,1752619,22,68412,"[""Eleanor McCutcheon Davis""]","[""graphic artist"",""animator"",""illustrator""]",http://www.wikidata.org/entity/Q5354261,http://www.wikidata.org/entity/Q1114448,Eleanor Davis,Cartoonist,317,9649,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14262,4165948,Netherlands,capital,Amsterdam,1825504,422,2455815,"[""Holland"",""the Netherlands"",""NL"",""NED"",""Neder...","[""Mokum"",""Amsterdam, NL"",""Amsterdam, Netherlan...",http://www.wikidata.org/entity/Q55,http://www.wikidata.org/entity/Q727,Netherlands,Amsterdam,416350,147710,What is the capital of Netherlands?,"[""The Hague"", ""Den Haag"", ""'s-Gravenhage"", ""Ha..."
14263,2617063,City Municipality of Celje,capital,Celje,1127153,422,5035,"[""Celje city municipality""]",[],http://www.wikidata.org/entity/Q3441823,http://www.wikidata.org/entity/Q1012,City Municipality of Celje,Celje,335,4086,What is the capital of City Municipality of Ce...,"[""Celje""]"
14264,471864,Delhi,capital,New Delhi,192624,422,2938512,"[""National Capital Region of Delhi"",""NCR""]","[""New Delhi district"",""Nayi Dilli""]",http://www.wikidata.org/entity/Q1353,http://www.wikidata.org/entity/Q987,Delhi,New Delhi,219227,128193,What is the capital of Delhi?,"[""New Delhi"", ""New Delhi district"", ""Nayi Dilli""]"
14265,2965573,Palestinian territories,capital,Jerusalem,1270077,422,127625,"[""occupied Palestinian territories"",""OPT"",""Wes...","[""Yerushalayim"",""J'lem"",""Aelia Capitolina"",""Al...",http://www.wikidata.org/entity/Q407199,http://www.wikidata.org/entity/Q1218,Palestinian territories,Jerusalem,17545,136734,What is the capital of Palestinian territories?,"[""Jerusalem"", ""Yerushalayim"", ""J'lem"", ""Aelia ..."


# Let's take a look at the output vector shape

In [34]:
for k, v in hs_cache_no_cot_other_all.items():
    print(f"{k} shape: {v[0].shape}")

ceval_liberal shape: torch.Size([746, 1536])
gsm8k shape: torch.Size([1319, 1536])
mgsm shape: torch.Size([1250, 1536])
popqa shape: torch.Size([14267, 1536])
gsm-symbolic shape: torch.Size([600, 1536])
human_eval shape: torch.Size([164, 1536])
mbpp shape: torch.Size([500, 1536])
mmlu-pro_600 shape: torch.Size([600, 1536])
mmlu-pro_3000 shape: torch.Size([3000, 1536])


# Let's take a look at the dataset

In [33]:
# pd.DataFrame(gsm8k_ds_main['train']).head()
pd.DataFrame(gsm_sym_data).head()

,id,instance,question,answer,original_id,original_question,original_answer,canary
0,0,0,Sanjay saw a 60-foot dolphin with 16 12-inch r...,"First, find the total number of remoras remain...",473,Benny saw a 10-foot shark with 2 6-inch remora...,"First, find the combined length of the remoras...",This document should not appear in training co...
1,0,1,Tara saw a 70-foot dolphin with 14 12-inch rem...,"First, find the total number of remoras remain...",473,Benny saw a 10-foot shark with 2 6-inch remora...,"First, find the combined length of the remoras...",This document should not appear in training co...
2,0,2,Liam saw a 50-foot shark with 12 15-inch remor...,"First, find the total number of remoras remain...",473,Benny saw a 10-foot shark with 2 6-inch remora...,"First, find the combined length of the remoras...",This document should not appear in training co...
3,0,3,Santiago saw a 80-foot dolphin with 12 24-inch...,"First, find the total number of remoras remain...",473,Benny saw a 10-foot shark with 2 6-inch remora...,"First, find the combined length of the remoras...",This document should not appear in training co...
4,0,4,Ananya saw a 70-foot dolphin with 14 12-inch r...,"First, find the total number of remoras remain...",473,Benny saw a 10-foot shark with 2 6-inch remora...,"First, find the combined length of the remoras...",This document should not appear in training co...


In [32]:
pd.DataFrame(other_dataset).head()

,task_id,text,code,test_list,test_setup_code,challenge_test_list
0,11,Write a python function to remove first and la...,"def remove_Occ(s,ch): \r\n for i in range(l...","[assert remove_Occ(""hello"",""l"") == ""heo"", asse...",,"[assert remove_Occ(""hellolloll"",""l"") == ""helol..."
1,12,Write a function to sort a given matrix in asc...,"def sort_matrix(M):\r\n result = sorted(M, ...","[assert sort_matrix([[1, 2, 3], [2, 4, 5], [1,...",,[]
2,13,Write a function to count the most common word...,from collections import Counter\r\ndef count_c...,"[assert count_common(['red','green','black','p...",,[]
3,14,Write a python function to find the volume of ...,"def find_Volume(l,b,h) : \r\n return ((l * ...","[assert find_Volume(10,8,6) == 240, assert fin...",,[]
4,15,Write a function to split a string at lowercas...,import re\r\ndef split_lowerstring(text):\r\n ...,"[assert split_lowerstring(""AbCd"")==['bC','d'],...",,[]


In [35]:
list(hs_cache_no_cot_other_all.keys())

['ceval_liberal',
 'gsm8k',
 'mgsm',
 'popqa',
 'gsm-symbolic',
 'human_eval',
 'mbpp',
 'mmlu-pro_600',
 'mmlu-pro_3000']

# save vectors

In [63]:
hs_cache_no_cot_other_all.keys()

dict_keys(['ceval_liberal', 'gsm8k', 'mgsm', 'popqa', 'mmlu-pro_3000', 'mmlu-pro_600', 'human_eval', 'mbpp', 'gsm_symbolic'])

In [58]:
hs_cache_no_cot_other_all['gsm_symbolic'] = hs_cache_no_cot_other_all['gsm-symbolic']

In [62]:
save_path = '../reasoning_representations_outputs'
os.makedirs(save_path, exist_ok=True)  

torch.save(hs_cache_no_cot_other_all, os.path.join(save_path, f'{model_name}-base_hs_cache_no_cot_all.pt'))